# POMCP

POMCP is one of the most widely-used online POMDP methods. It is a variant of Monte Carlo tree search in which each node corresponds to an action-observation history.

(To get the required packages for this notebook, make sure to run the `install.jl` script)

In [1]:
using POMDPModels # for the crying baby problem
using BasicPOMCP
using Interact
rng = MersenneTwister(7);

In [2]:
pomdp = BabyPOMDP()
b = initial_state_distribution(pomdp);

Since POMCP is an online method, it does no offline computation before being deployed into the environment. Thus the `solve` function from `POMDPs.jl` does not computation, but instead creates a planner optimized for the problem.

In [3]:
solver = POMCPSolver(c=10.0); # c is the exploration constant
planner = solve(solver, pomdp);

## Building the tree

To make a decision, POMCP constructs a tree by repeating the following process:
1. run a simulation through the tree choosing actions based on the *UCB criterion*
2. when a leaf node is reached, expand the node one level deeper and continue the simulation with a *rollout policy* (often random)
3. propagate the rewards back up the tree to maintain an estimate of the *value*\* for each action

\*This is denoted by *V* in the `BasicPOMCP` package and the original paper that describes POMCP, but it is the same thing as the state-action value typically denoted by *Q*.

In [4]:
N = 10
tree = BasicPOMCP.POMCPTree(pomdp)
trees = [deepcopy(tree)]
for i in 1:N
    s = rand(rng, b)
    BasicPOMCP.simulate(planner, s, BasicPOMCP.POMCPObsNode(tree, 1), 10)
    push!(trees, deepcopy(tree))
end

In [5]:
@manipulate for i in 1:length(trees)
    D3Tree(trees[i], init_expand=100, init_duration=0)
end

Interact.Options{:SelectionSlider,Any}(1: "input" = 6 Any , "i", 6, "6", 6, Interact.OptionDict(DataStructures.OrderedDict{Any,Any}("1"=>1,"2"=>2,"3"=>3,"4"=>4,"5"=>5,"6"=>6,"7"=>7,"8"=>8,"9"=>9,"10"=>10…), Dict{Any,Any}(Pair{Any,Any}(2, "2"),Pair{Any,Any}(11, "11"),Pair{Any,Any}(7, "7"),Pair{Any,Any}(9, "9"),Pair{Any,Any}(10, "10"),Pair{Any,Any}(8, "8"),Pair{Any,Any}(6, "6"),Pair{Any,Any}(4, "4"),Pair{Any,Any}(3, "3"),Pair{Any,Any}(5, "5")…)), Any[], Any[], true, "horizontal")

o: <root>
N: 5         

├──a: true
│  N: 2       V: -19.8     
│  └──o: false
│     N: 1         

│     ├──a: true
│     │  N: 0       V: 0          (0 children)
│     └──a: false
│        N: 1       V: -11.5      (1 children)
└──a: false
   N: 3       V: -21.7     
   └──o: false
      N: 2         

      ├──a: true
      │  N: 1       V: -25.5      (1 children)
      └──a: false
         N: 1       V: -20.8      (1 children)

## Online operation

When the agent is interacting with the environment, it constructs a new tree at each step, makes a decision. For this demonstation, we'll use 

In [6]:
using POMDPs
using LaserTag
rng = MersenneTwister(14)
laser = gen_lasertag(rng=rng)
s = initial_state(laser, rng)
sp,o,r = generate_sor(laser, s, 1, rng)
HTML("""<div style="width: 400px; margin: 0 auto">
    $(stringmime("image/svg+xml", LaserTagVis(laser, s=sp, o=o)))
        </div>
    """)

HTML{String}("<div style=\"width: 400px; margin: 0 auto\">\n<?xml version=\"1.0\" encoding=\"UTF-8\"?>\n<svg xmlns=\"http://www.w3.org/2000/svg\" xmlns:xlink=\"http://www.w3.org/1999/xlink\" width=\"389.764pt\" height=\"248.031pt\" viewBox=\"0 0 389.764 248.031\" version=\"1.1\">\n<defs>\n<clipPath id=\"clip-1509070005248971-1\">\n  <path d=\"M 35 212 L 71 212 L 71 248.03125 L 35 248.03125 Z M 35 212 \"/>\n</clipPath>\n</defs>\n<g id=\"surface1\">\n<g clip-path=\"url(#clip-1509070005248971-1)\" clip-rule=\"nonzero\">\n<path style=\" stroke:none;fill-rule:nonzero;fill:rgb(50%,50%,50%);fill-opacity:0.5;\" d=\"M 35.433594 248.03125 L 35.433594 212.597656 L 70.867188 212.597656 L 70.867188 248.03125 Z M 35.433594 248.03125 \"/>\n</g>\n<path style=\" stroke:none;fill-rule:nonzero;fill:rgb(50%,50%,50%);fill-opacity:0.5;\" d=\"M 318.902344 177.164062 L 318.902344 141.730469 L 354.335938 141.730469 L 354.335938 177.164062 Z M 318.902344 177.164062 \"/>\n<path style=\" stroke:none;fill-rule:nonzero;fill:rgb(50%,50%,50%);fill-opacity:0.5;\" d=\"M 177.167969 177.164062 L 177.167969 141.730469 L 212.601562 141.730469 L 212.601562 177.164062 Z M 177.167969 177.164062 \"/>\n<path style=\" stroke:none;fill-rule:nonzero;fill:rgb(50%,50%,50%);fill-opacity:0.5;\" d=\"M 0 212.597656 L 0 177.164062 L 35.433594 177.164062 L 35.433594 212.597656 Z M 0 212.597656 \"/>\n<path style=\" stroke:none;fill-rule:nonzero;fill:rgb(50%,50%,50%);fill-opacity:0.5;\" d=\"M 0 106.296875 L 0 70.863281 L 35.433594 70.863281 L 35.433594 106.296875 Z M 0 106.296875 \"/>\n<path style=\" stroke:none;fill-rule:nonzero;fill:rgb(50%,50%,50%);fill-opacity:0.5;\" d=\"M 141.734375 106.296875 L 141.734375 70.863281 L 177.167969 70.863281 L 177.167969 106.296875 Z M 141.734375 106.296875 \"/>\n<path style=\" stroke:none;fill-rule:nonzero;fill:rgb(50%,50%,50%);fill-opacity:0.5;\" d=\"M 177.167969 70.863281 L 177.167969 35.429688 L 212.601562 35.429688 L 212.601562 70.863281 Z M 177.167969 70.863281 \"/>\n<path style=\" stroke:none;fill-rule:nonzero;fill:rgb(50%,50%,50%);fill-opacity:0.5;\" d=\"M 141.734375 70.863281 L 141.734375 35.429688 L 177.167969 35.429688 L 177.167969 70.863281 Z M 141.734375 70.863281 \"/>\n<path style=\" stroke:none;fill-rule:nonzero;fill:rgb(100%,50%,0%);fill-opacity:0.5;\" d=\"M 318.902344 141.730469 L 318.902344 106.296875 L 354.335938 106.296875 L 354.335938 141.730469 Z M 318.902344 141.730469 \"/>\n<path style=\" stroke:none;fill-rule:nonzero;fill:rgb(0%,100%,0%);fill-opacity:0.5;\" d=\"M 212.601562 106.296875 L 212.601562 70.863281 L 248.035156 70.863281 L 248.035156 106.296875 Z M 212.601562 106.296875 \"/>\n<path style=\"fill:none;stroke-width:0.3985;stroke-linecap:butt;stroke-linejoin:miter;stroke:rgb(100%,0%,0%);stroke-opacity:1;stroke-dasharray:2.98883,2.98883;stroke-miterlimit:10;\" d=\"M 230.316406 177.167719 L 230.316406 177.167719 \" transform=\"matrix(1,0,0,-1,0,248.031)\"/>\n<path style=\"fill:none;stroke-width:0.3985;stroke-linecap:butt;stroke-linejoin:miter;stroke:rgb(100%,0%,0%);stroke-opacity:1;stroke-dasharray:2.98883,2.98883;stroke-miterlimit:10;\" d=\"M 248.035156 159.448969 L 318.902344 159.448969 \" transform=\"matrix(1,0,0,-1,0,248.031)\"/>\n<path style=\"fill:none;stroke-width:0.3985;stroke-linecap:butt;stroke-linejoin:miter;stroke:rgb(100%,0%,0%);stroke-opacity:1;stroke-dasharray:2.98883,2.98883;stroke-miterlimit:10;\" d=\"M 230.316406 141.734125 L 230.316406 70.866938 \" transform=\"matrix(1,0,0,-1,0,248.031)\"/>\n<path style=\"fill:none;stroke-width:0.3985;stroke-linecap:butt;stroke-linejoin:miter;stroke:rgb(100%,0%,0%);stroke-opacity:1;stroke-dasharray:2.98883,2.98883;stroke-miterlimit:10;\" d=\"M 212.601562 159.448969 L 177.167969 159.448969 \" transform=\"matrix(1,0,0,-1,0,248.031)\"/>\n<path style=\"fill:none;stroke-width:0.3985;stroke-linecap:butt;stroke-linejoin:miter;stroke:rgb(100%,0%,0%);stroke-opacity:1;stroke-dasharray:2.98883,2.98883;stroke-miterlimit:10;\" d=\"M 248.035156 177.167719 L 298.136719 227.269281 \

In [7]:
using DiscreteValueIteration # value iteration is used instead of rollouts
using POMDPToolbox
using BasicPOMCP
solver = POMCPSolver(tree_queries=1000,
                      c=20.0,
                      max_depth=100,
                      estimate_value=FOValue(ValueIterationSolver()),
                      rng=MersenneTwister(13)
                     )
planner = solve(solver, laser);

In [8]:
tree_recorder = PolicyWrapper(planner, payload=[]) do planner, payload, b
    a = action(planner, b)
    push!(payload, get(planner._tree))
    return a
end

sim = HistoryRecorder(max_steps=100, show_progress=true, rng=MersenneTwister(30))
history = simulate(sim, laser, tree_recorder);

Simulating...100%|██████████████████████████████████████| Time: 0:00:01


In [9]:
using Interact
using D3Trees
value = Interact.value
ltrees = tree_recorder.payload
steps = eachstep(history, "a,r,sp,o,bp")
BasicPOMCP.node_tag(a::Int) = LaserTag.ACTION_NAMES[a] 
@manipulate for i in 1:length(trees)
    ltree = stringmime("text/html", D3Tree(ltrees[i],
                                          init_duration=0,
                                          init_expand=1,
                                          svg_height=300))
    pic = stringmime("image/svg+xml", LaserTagVis(laser, steps[i]...))
    HTML("""<div style="width: 300px; margin: 0 auto">$pic</div><div>$ltree</div>""")
end

Interact.Options{:SelectionSlider,Any}(5: "input-2" = 6 Any , "i", 6, "6", 6, Interact.OptionDict(DataStructures.OrderedDict{Any,Any}("1"=>1,"2"=>2,"3"=>3,"4"=>4,"5"=>5,"6"=>6,"7"=>7,"8"=>8,"9"=>9,"10"=>10…), Dict{Any,Any}(Pair{Any,Any}(2, "2"),Pair{Any,Any}(11, "11"),Pair{Any,Any}(7, "7"),Pair{Any,Any}(9, "9"),Pair{Any,Any}(10, "10"),Pair{Any,Any}(8, "8"),Pair{Any,Any}(6, "6"),Pair{Any,Any}(4, "4"),Pair{Any,Any}(3, "3"),Pair{Any,Any}(5, "5")…)), Any[], Any[], true, "horizontal")

HTML{String}("<div style=\"width: 300px; margin: 0 auto\"><?xml version=\"1.0\" encoding=\"UTF-8\"?>\n<svg xmlns=\"http://www.w3.org/2000/svg\" xmlns:xlink=\"http://www.w3.org/1999/xlink\" width=\"389.764pt\" height=\"248.031pt\" viewBox=\"0 0 389.764 248.031\" version=\"1.1\">\n<defs>\n<g>\n<symbol overflow=\"visible\" id=\"glyph-1509070005248972-0-0\">\n<path style=\"stroke:none;\" d=\"\"/>\n</symbol>\n<symbol overflow=\"visible\" id=\"glyph-1509070005248972-0-1\">\n<path style=\"stroke:none;\" d=\"M 6.171875 -3.34375 C 6.34375 -3.84375 6.65625 -3.984375 7.015625 -3.984375 L 7.015625 -4.296875 C 6.78125 -4.28125 6.5 -4.265625 6.28125 -4.265625 C 5.984375 -4.265625 5.546875 -4.28125 5.359375 -4.296875 L 5.359375 -3.984375 C 5.71875 -3.984375 5.9375 -3.796875 5.9375 -3.515625 C 5.9375 -3.453125 5.9375 -3.421875 5.875 -3.296875 L 4.96875 -0.75 L 3.984375 -3.53125 C 3.953125 -3.65625 3.9375 -3.671875 3.9375 -3.71875 C 3.9375 -3.984375 4.328125 -3.984375 4.53125 -3.984375 L 4.53125 -4.296875 C 4.234375 -4.28125 3.734375 -4.265625 3.484375 -4.265625 C 3.1875 -4.265625 2.90625 -4.28125 2.609375 -4.296875 L 2.609375 -3.984375 C 2.96875 -3.984375 3.125 -3.96875 3.234375 -3.84375 C 3.28125 -3.78125 3.390625 -3.484375 3.453125 -3.296875 L 2.609375 -0.875 L 1.65625 -3.53125 C 1.609375 -3.65625 1.609375 -3.671875 1.609375 -3.71875 C 1.609375 -3.984375 2 -3.984375 2.1875 -3.984375 L 2.1875 -4.296875 C 1.890625 -4.28125 1.328125 -4.265625 1.109375 -4.265625 C 1.0625 -4.265625 0.53125 -4.28125 0.171875 -4.296875 L 0.171875 -3.984375 C 0.671875 -3.984375 0.796875 -3.953125 0.921875 -3.640625 L 2.171875 -0.109375 C 2.21875 0.03125 2.25 0.109375 2.375 0.109375 C 2.515625 0.109375 2.53125 0.046875 2.578125 -0.09375 L 3.59375 -2.90625 L 4.609375 -0.078125 C 4.640625 0.03125 4.671875 0.109375 4.8125 0.109375 C 4.9375 0.109375 4.96875 0.015625 5 -0.078125 Z M 6.171875 -3.34375 \"/>\n</symbol>\n<symbol overflow=\"visible\" id=\"glyph-1509070005248972-0-2\">\n<path style=\"stroke:none;\" d=\"M 1.109375 -2.515625 C 1.171875 -4 2.015625 -4.25 2.359375 -4.25 C 3.375 -4.25 3.484375 -2.90625 3.484375 -2.515625 Z M 1.109375 -2.296875 L 3.890625 -2.296875 C 4.109375 -2.296875 4.140625 -2.296875 4.140625 -2.515625 C 4.140625 -3.5 3.59375 -4.46875 2.359375 -4.46875 C 1.203125 -4.46875 0.28125 -3.4375 0.28125 -2.1875 C 0.28125 -0.859375 1.328125 0.109375 2.46875 0.109375 C 3.6875 0.109375 4.140625 -1 4.140625 -1.1875 C 4.140625 -1.28125 4.0625 -1.3125 4 -1.3125 C 3.921875 -1.3125 3.890625 -1.25 3.875 -1.171875 C 3.53125 -0.140625 2.625 -0.140625 2.53125 -0.140625 C 2.03125 -0.140625 1.640625 -0.4375 1.40625 -0.8125 C 1.109375 -1.28125 1.109375 -1.9375 1.109375 -2.296875 Z M 1.109375 -2.296875 \"/>\n</symbol>\n<symbol overflow=\"visible\" id=\"glyph-1509070005248972-0-3\">\n<path style=\"stroke:none;\" d=\"M 2.078125 -1.9375 C 2.296875 -1.890625 3.109375 -1.734375 3.109375 -1.015625 C 3.109375 -0.515625 2.765625 -0.109375 1.984375 -0.109375 C 1.140625 -0.109375 0.78125 -0.671875 0.59375 -1.53125 C 0.5625 -1.65625 0.5625 -1.6875 0.453125 -1.6875 C 0.328125 -1.6875 0.328125 -1.625 0.328125 -1.453125 L 0.328125 -0.125 C 0.328125 0.046875 0.328125 0.109375 0.4375 0.109375 C 0.484375 0.109375 0.5 0.09375 0.6875 -0.09375 C 0.703125 -0.109375 0.703125 -0.125 0.890625 -0.3125 C 1.328125 0.09375 1.78125 0.109375 1.984375 0.109375 C 3.125 0.109375 3.59375 -0.5625 3.59375 -1.28125 C 3.59375 -1.796875 3.296875 -2.109375 3.171875 -2.21875 C 2.84375 -2.546875 2.453125 -2.625 2.03125 -2.703125 C 1.46875 -2.8125 0.8125 -2.9375 0.8125 -3.515625 C 0.8125 -3.875 1.0625 -4.28125 1.921875 -4.28125 C 3.015625 -4.28125 3.078125 -3.375 3.09375 -3.078125 C 3.09375 -2.984375 3.1875 -2.984375 3.203125 -2.984375 C 3.34375 -2.984375 3.34375 -3.03125 3.34375 -3.21875 L 3.34375 -4.234375 C 3.34375 -4.390625 3.34375 -4.46875 3.234375 -4.46875 C 3.1875 -4.46875 3.15625 -4.46875 3.03125 -4.34375 C 3 -4.3125 2.90625 -4.21875 2.859375 -4.1875 C 2.484375 -4.46875 2.078125 -4.46875 1.921875 -4.468